In [ ]:
pip install fsspec

In [ ]:
pip install --upgrade huggingface_hub

In [ ]:
pip install torch

In [1]:
import pandas as pd
import re
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
import torch.optim as optim
import random
import torch.nn.functional as F

In [2]:
# Load the dataset from Hugging Face
df = pd.read_json("hf://datasets/b-mc2/sql-create-context/sql_create_context_v4.json")

# Display the first few rows of the dataset
print(df.head())

/Users/mukesh/Library/Python/3.12/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


                                            question  \
0  How many heads of the departments are older th...   
1  List the name, born state and age of the heads...   
2  List the creation year, name and budget of eac...   
3  What are the maximum and minimum budget of the...   
4  What is the average number of employees of the...   

                                             context  \
0                    CREATE TABLE head (age INTEGER)   
1  CREATE TABLE head (name VARCHAR, born_state VA...   
2  CREATE TABLE department (creation VARCHAR, nam...   
3  CREATE TABLE department (budget_in_billions IN...   
4  CREATE TABLE department (num_employees INTEGER...   

                                              answer  
0           SELECT COUNT(*) FROM head WHERE age > 56  
1  SELECT name, born_state, age FROM head ORDER B...  
2  SELECT creation, name, budget_in_billions FROM...  
3  SELECT MAX(budget_in_billions), MIN(budget_in_...  
4  SELECT AVG(num_employees) FROM department WHER..

In [3]:
# Custom Dataset for natural language prompt and SQL pairs
class NL2SQLDataset(Dataset):
    def __init__(self, df):
        self.prompts = df['question'].tolist()
        self.sql_queries = df['answer'].tolist()
        self.vocab = self.build_vocab()

    def build_vocab(self):
        tokens = set()
        for prompt, sql in zip(self.prompts, self.sql_queries):
            tokens.update(self.tokenize(prompt))
            tokens.update(self.tokenize(sql))
        # Add special tokens for padding and end-of-sequence (EOS)
        tokens.update(['<PAD>', '<EOS>'])
        return {token: i for i, token in enumerate(sorted(tokens))}

    def tokenize(self, text):
        # Tokenizer splitting by spaces and punctuation
        return re.findall(r"[\w']+|[.,!?;]", text)

    def encode(self, text):
        tokens = self.tokenize(text)
        return [self.vocab[token] for token in tokens]

    def __len__(self):
        return len(self.prompts)

    def __getitem__(self, idx):
        prompt = self.encode(self.prompts[idx])
        sql = self.encode(self.sql_queries[idx]) + [self.vocab['<EOS>']]  # Add EOS token at the end
        return torch.tensor(prompt), torch.tensor(sql)

# Custom collate function to pad sequences to the same length
def collate_fn(batch):
    prompts, sqls = zip(*batch)
    
    # Pad the sequences with the padding token
    padded_prompts = pad_sequence(prompts, batch_first=True, padding_value=dataset.vocab['<PAD>'])
    padded_sqls = pad_sequence(sqls, batch_first=True, padding_value=dataset.vocab['<PAD>'])
    
    return padded_prompts, padded_sqls

# Load the dataset
dataset = NL2SQLDataset(df)
# Load the dataset with the custom collate function
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

In [4]:
# Define the Transformer model
class TransformerModel(nn.Module):
    def __init__(self, input_dim, output_dim, d_model=512, nhead=8, num_layers=6):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Embedding(input_dim, d_model)
        self.transformer = nn.Transformer(d_model=d_model, nhead=nhead, num_encoder_layers=num_layers, num_decoder_layers=num_layers)
        self.fc_out = nn.Linear(d_model, output_dim)

    def forward(self, src, tgt):
        # Embed both the source and target
        src_emb = self.embedding(src)
        tgt_emb = self.embedding(tgt)

        # Transformer expects input of shape [sequence_length, batch_size, d_model]
        src_emb = src_emb.transpose(0, 1)  # Transpose to [sequence_length, batch_size, d_model]
        tgt_emb = tgt_emb.transpose(0, 1)  # Transpose to [sequence_length, batch_size, d_model]

        # Pass through transformer and then final linear layer
        output = self.transformer(src_emb, tgt_emb)
        output = self.fc_out(output)

        # Output shape [sequence_length, batch_size, vocab_size] -> transpose to [batch_size, sequence_length, vocab_size]
        return output.transpose(0, 1)

# Initialize the model
input_dim = len(dataset.vocab)
output_dim = len(dataset.vocab)
model = TransformerModel(input_dim, output_dim)

/opt/homebrew/Cellar/jupyterlab/4.0.8_1/libexec/lib/python3.12/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [ ]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=dataset.vocab['<PAD>'])  # Ignore padding in loss calculation
optimizer = optim.Adam(model.parameters())

# Training loop
def train(model, dataloader, epochs=20):
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        for batch in dataloader:
            src, tgt = batch
            optimizer.zero_grad()
            
            # Target input shifted for teacher forcing
            output = model(src, tgt[:, :-1])
            loss = criterion(output.reshape(-1, output_dim), tgt[:, 1:].reshape(-1))  # Shift target by 1 for the loss calculation
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(dataloader)}")

# Train the model
train(model, dataloader)

Epoch 1/20, Loss: 6.652109950877945
Epoch 2/20, Loss: 6.478104196464589
Epoch 3/20, Loss: 6.466616767431315
Epoch 4/20, Loss: 6.461856329868205
Epoch 5/20, Loss: 6.456198759110044
Epoch 6/20, Loss: 6.452568362124191
Epoch 7/20, Loss: 6.44982899730291


In [ ]:
# Function to sample from the top-k tokens during generation
def top_k_sampling(logits, k=10):
    # Get the top-k logits and their corresponding indices
    top_k_logits, top_k_indices = torch.topk(logits, k, dim=-1)
    
    # Apply softmax to get probabilities and sample from the top-k
    top_k_probs = F.softmax(top_k_logits, dim=-1)
    next_token = random.choices(top_k_indices.squeeze().tolist(), top_k_probs.squeeze().tolist())[0]
    
    return next_token

In [ ]:
# Updated generate_sql function with minimum token generation length
def generate_sql(model, prompt, max_len=100, k=10, min_len=5):
    model.eval()
    tokens = dataset.encode(prompt)
    generated = []
    src = torch.tensor(tokens).unsqueeze(0)  # Add batch dimension
    tgt = torch.tensor([dataset.vocab['<PAD>']]).unsqueeze(0)  # Initial target input

    for i in range(max_len):
        output = model(src, tgt)
        next_token_logits = output[:, -1, :]  # Get logits for the last timestep
        next_token = top_k_sampling(next_token_logits, k)  # Sample using top-k
        
        generated.append(next_token)
        
        # Prevent the model from ending the sequence too early
        if next_token == dataset.vocab['<EOS>'] and len(generated) < min_len:
            continue  # Ignore EOS if less than min_len tokens are generated
        
        # Update target sequence
        tgt = torch.cat([tgt, torch.tensor([[next_token]])], dim=1)
        
        if next_token == dataset.vocab['<EOS>']:
            break

    return generated

In [ ]:
# Main loop for user input
while True:
    user_prompt = input("Enter your query in natural language (or 'exit' to quit): ")
    if user_prompt.lower() == 'exit':
        break

    # Generate SQL query based on user input
    sql_tokens = generate_sql(model, user_prompt)
    sql_query = ' '.join([list(dataset.vocab.keys())[list(dataset.vocab.values()).index(token)] for token in sql_tokens])
    
    print(f"Generated SQL Query: {sql_query}") 